<a href="https://colab.research.google.com/github/omsin/NeoCP/blob/main/NeoCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! cp /content/drive/MyDrive/dataset/finalDataset_0208.csv /content

In [ ]:
! nvidia-smi

Thu Apr 14 12:08:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install datasets transformers -q

In [ ]:
!pip3 install wandb --upgrade -q

In [ ]:
!pip3 install fast_ml

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

from datasets import load_dataset
from sklearn.model_selection import train_test_split
import random
from sklearn import metrics, model_selection, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from fast_ml.model_development import train_valid_test_split
from transformers import Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
"""#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='admin', password='YzGpGi9Jrp', host='ec2-35-164-255-171.us-west-2.compute.amazonaws.com', port= '5432'
)
#Setting auto commit false
conn.autocommit = True

#Query data into dataframe
table = pd.read_sql_query("SELECT * from gold_news", conn)

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

table.head()"""

'#establishing the connection\nconn = psycopg2.connect(\n   database="postgres", user=\'admin\', password=\'YzGpGi9Jrp\', host=\'ec2-35-164-255-171.us-west-2.compute.amazonaws.com\', port= \'5432\'\n)\n#Setting auto commit false\nconn.autocommit = True\n\n#Query data into dataframe\ntable = pd.read_sql_query("SELECT * from gold_news", conn)\n\n#Commit your changes in the database\nconn.commit()\n\n#Closing the connection\nconn.close()\n\ntable.head()'

In [ ]:
data = pd.read_csv("/content/finalDataset_0208.csv") 
data.head()

,Dates,URL,News,Price or Not,Direction Up,Direction Constant,Direction Down,PastPrice,FuturePrice,PastNews,FutureNews,Asset Comparision
0,2016-01-28,http://www.marketwatch.com/story/april-gold-do...,"april gold down 20 cents to settle at $1,116.1...",1,0,0,1,1,0,0,0,0
1,2017-09-13,http://www.marketwatch.com/story/gold-prices-s...,gold suffers third straight daily decline,1,0,0,1,1,0,0,0,0
2,2016-07-26,http://www.marketwatch.com/story/gold-futures-...,Gold futures edge up after two-session decline,1,1,0,0,1,0,0,0,0
3,2018-02-28,https://www.metalsdaily.com/link/277199/dent-r...,dent research : is gold’s day in the sun comin...,0,0,0,0,0,0,0,1,0
4,2017-09-06,http://www.marketwatch.com/story/gold-steadies...,"Gold snaps three-day rally as Trump, lawmakers...",1,1,0,0,1,0,0,0,0


In [ ]:
data.columns = ['Dates', 'URL', 'News', 0, 1, 2, 3, 4, 5, 6, 7, 8]
data.head()

,Dates,URL,News,0,1,2,3,4,5,6,7,8
0,2016-01-28,http://www.marketwatch.com/story/april-gold-do...,"april gold down 20 cents to settle at $1,116.1...",1,0,0,1,1,0,0,0,0
1,2017-09-13,http://www.marketwatch.com/story/gold-prices-s...,gold suffers third straight daily decline,1,0,0,1,1,0,0,0,0
2,2016-07-26,http://www.marketwatch.com/story/gold-futures-...,Gold futures edge up after two-session decline,1,1,0,0,1,0,0,0,0
3,2018-02-28,https://www.metalsdaily.com/link/277199/dent-r...,dent research : is gold’s day in the sun comin...,0,0,0,0,0,0,0,1,0
4,2017-09-06,http://www.marketwatch.com/story/gold-steadies...,"Gold snaps three-day rally as Trump, lawmakers...",1,1,0,0,1,0,0,0,0


# **Training**

In [ ]:
def seed_everything(seed=73):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [ ]:
import wandb

wandb.login()

wandb: Currently logged in as: omsin (use `wandb login --relogin` to force relogin)


True

In [ ]:
sweep_config = {
    'method': 'random', #grid, random, bayesian
    'metric': {
      'name': 'auc_score',
      'goal': 'maximize'   
    },
    'parameters': {

        'learning_rate': {
            'values': [5e-5, 3e-5]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'epochs':{'value': 10},
        'dropout':{
            'values': [0.3, 0.4, 0.5]
        },
        'tokenizer_max_len': {'value': 40},
    }
}

sweep_defaults = {
    'learning_rate': 3e-5,
    'batch_size': 64,
    'epochs': 10,
    'dropout': 0.3,
    'tokenizer_max_len': 40
}

sweep_id = wandb.sweep(sweep_config, project='NeoCP1')

Create sweep with ID: 3svy686k
Sweep URL: https://wandb.ai/omsin/NeoCP1/sweeps/3svy686k


In [ ]:
train, tests = train_test_split(data, train_size=0.8)

In [ ]:
valid, test = train_test_split(data, train_size=0.5)

In [ ]:
print(train.shape, valid.shape, test.shape) 

(9129, 12) (5706, 12) (5706, 12)


In [ ]:
n_labels = 9

In [ ]:
class GoNeoCPDataset:
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [ ]:
class GoNeoCPClassifier(nn.Module):
    def __init__(self, n_train_steps, n_classes, do_prob, bert_model):
        super(GoNeoCPClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(do_prob)
        self.out = nn.Linear(768, n_classes)
        self.n_train_steps = n_train_steps
        self.step_scheduler_after = "batch"

    def forward(self, ids, mask):
        output_1 = self.bert(ids, attention_mask=mask)["pooler_output"]
        output_2 = self.dropout(output_1)
        output = self.out(output_2)
        return output

In [ ]:
tokenizer = transformers.SqueezeBertTokenizer.from_pretrained("squeezebert/squeezebert-uncased", do_lower_case=True)

def build_dataset(tokenizer_max_len):
    train_dataset = GoNeoCPDataset(train.News.tolist(), train[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    valid_dataset = GoNeoCPDataset(valid.News.tolist(), valid[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    
    return train_dataset, valid_dataset

def build_dataloader(train_dataset, valid_dataset, batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_data_loader, valid_data_loader

def ret_model(n_train_steps, do_prob):
  model = GoNeoCPClassifier(n_train_steps, n_labels, do_prob, bert_model=bert_model)
  return model

In [ ]:
sample_train_dataset, _ = build_dataset(20)
print(sample_train_dataset[50])
len(sample_train_dataset)

{'ids': tensor([  101,  2751,  7597,  2000,  3119,  2896,  2651,  1024,  4850, 21955,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'labels': tensor([1, 0, 0, 1, 0, 1, 0, 0, 0])}


9129

In [ ]:
bert_model = transformers.SqueezeBertModel.from_pretrained("squeezebert/squeezebert-uncased")

Some weights of the model checkpoint at squeezebert/squeezebert-uncased were not used when initializing SqueezeBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing SqueezeBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SqueezeBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def ret_optimizer(model):
    '''
    Taken from Abhishek Thakur's Tez library example: 
    https://github.com/abhishekkrthakur/tez/blob/main/examples/text_classification/binary.py
    '''
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    opt = AdamW(optimizer_parameters, lr=wandb.config.learning_rate)
    return opt

def ret_scheduler(optimizer, num_train_steps):
    sch = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return sch

def loss_fn(outputs, labels):
    if labels is None:
        return None
    return nn.BCEWithLogitsLoss()(outputs, labels.float())

def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    '''
    auc_micro_list = []
    for i in range(n_labels):
      current_pred = preds.T[i]
      current_label = labels.T[i]
      fpr_micro, tpr_micro, _ = metrics.roc_curve(current_label.T, current_pred.T)
      auc_micro = metrics.auc(fpr_micro, tpr_micro)
      auc_micro_list.append(auc_micro)
    
    return {"auc": np.array(auc_micro).mean()}
    '''

    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    return {"auc_micro": auc_micro}

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    '''
        Modified from Abhishek Thakur's BERT example: 
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''

    train_loss = 0.0
    model.train()
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask)

        loss = loss_fn(outputs, targets)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        scheduler.step()
    return train_loss
    

def eval_fn(data_loader, model, device):
    '''
        Modified from Abhishek Thakur's BERT example: 
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''
    eval_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask)
            loss = loss_fn(outputs, targets)
            eval_loss += loss.item()
            fin_targets.extend(targets)
            fin_outputs.extend(torch.sigmoid(outputs))
    return eval_loss, fin_outputs, fin_targets

In [ ]:
def trainer(config=None):
    with wandb.init(config=config):
        config = wandb.config

        train_dataset, valid_dataset = build_dataset(config.tokenizer_max_len)
        train_data_loader, valid_data_loader = build_dataloader(train_dataset, valid_dataset, config.batch_size)
        print("Length of Train Dataloader: ", len(train_data_loader))
        print("Length of Valid Dataloader: ", len(valid_data_loader))

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        n_train_steps = int(len(train_dataset) / config.batch_size * 10)

        model = ret_model(n_train_steps, config.dropout)
        optimizer = ret_optimizer(model)
        scheduler = ret_scheduler(optimizer, n_train_steps)
        model.to(device)
        model = nn.DataParallel(model)
        wandb.watch(model)
        
        n_epochs = config.epochs

        best_val_loss = 100
        for epoch in tqdm(range(n_epochs)):
            train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
            eval_loss, preds, labels = eval_fn(valid_data_loader, model, device)
          
            auc_score = log_metrics(preds, labels)["auc_micro"]
            print("AUC score: ", auc_score)
            avg_train_loss, avg_val_loss = train_loss / len(train_data_loader), eval_loss / len(valid_data_loader)
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                "val_loss": avg_val_loss,
                "auc_score": auc_score,
            })
            print("Average Train loss: ", avg_train_loss)
            print("Average Valid loss: ", avg_val_loss)

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save(model.state_dict(), "./best_model.pt")  
                print("Model saved as current val_loss is: ", best_val_loss)    

In [ ]:
wandb.agent(sweep_id, function=trainer, count=6)

wandb: Agent Starting Run: 2hhyz0xe with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	learning_rate: 5e-05
wandb: 	tokenizer_max_len: 40
wandb: Currently logged in as: omsin (use `wandb login --relogin` to force relogin)


Length of Train Dataloader:  143
Length of Valid Dataloader:  90


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/143 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

...

AUC score:  0.9976418647624303
Average Train loss:  0.008517975400371308
Average Valid loss:  0.029802423739264713
Model saved as current val_loss is:  0.029802423739264713


  0%|          | 0/286 [00:00<?, ?it/s]

  0%|          | 0/179 [00:00<?, ?it/s]

AUC score:  0.997674715011025
Average Train loss:  0.008250775319602261
Average Valid loss:  0.031149327723756847



auc_score,▂▆▄▁▇▇██▇▇
epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▃▂▂▁▁▁▁▁▁
val_loss,█▃▃▂▁▁▁▁▁▁
auc_score,0.99767
epoch,10
train_loss,0.00825
val_loss,0.03115


In [ ]:
! cp /content/best_model.pt /content/drive/MyDrive/model

# **Testing**

In [ ]:
! cp /content/drive/MyDrive/model/best_model.pt /content

In [ ]:
class GoNeoCPTestDataset:
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts

        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long)
        }

In [ ]:
class GoNeoCPDataset2:
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "label": torch.tensor(label, dtype=torch.long)
        }

In [ ]:
test_list = test.News.tolist()
# n_train_steps = int(len(train_dataset) / 32 * 10)
model = GoNeoCPClassifier(28, 9, 0.5, bert_model=bert_model)
#model = torch.load('/content/best_model.pt')
model.load_state_dict(torch.load('/content/best_model.pt'), strict=False)
# model.load_state_dict(torch.load('/content/best_model.pt'))
#model.eval()
#model.load('/content/best_model.pt')
#predicted = bert_model.predict(test_list)
#test_dataset = GoNeoCPDataset2(test.News.tolist(),test[range(n_labels)].values.tolist(), tokenizer, max_len=20)
test_dataset = GoNeoCPTestDataset(test.News.tolist(),tokenizer, max_len=20)
testdata = []
mask = []
for i in range(len(test_dataset)):
  testdata.append(test_dataset[i]['ids'].tolist())
  mask.append(test_dataset[i]['mask'].tolist())
cuda0 = torch.device('cuda:0')
testdata = torch.tensor(testdata, device=cuda0)
mask = torch.tensor(mask, device=cuda0)
print(testdata)
prediction = model(testdata[0], mask[0])

tensor([[  101,  2047,  3513,  ...,     0,     0,     0],
        [  101,  2751, 18881,  ...,     0,     0,     0],
        [  101,  2859,  7457,  ...,     0,     0,     0],
        ...,
        [  101,  2257,  2751,  ...,  1012,  3938,   102],
        [  101,  2751, 17795,  ...,     0,     0,     0],
        [  101,  2751,  7822,  ...,     0,     0,     0]], device='cuda:0')


ValueError: ignored

In [ ]:
for in_ids in testdata:
	testdata.resize_(1,len(in_ids))

In [ ]:
testdata

tensor([[  101,  2047,  3513,  2005,  2751, 19920,  2089,  2330,  1996,  3006,
          2000,  1015,  1012,  1020,  4551,  7486,   102,     0,     0,     0]],
       device='cuda:0')

In [ ]:
print(test_dataset[3])
len(test_dataset)

{'ids': tensor([  101, 13336,  1997,  1037, 13007,  4667,  2751,  8320,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'label': tensor([0, 0, 0, 0, 0, 0, 1, 0, 0])}


5706

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 5706
  Batch size = 8


ValueError: ignored